# First NGSolve example (Unit 1.1)

Let us solve the Poisson problem.
$$\text{find: } u \in H_{0,D}^1 \int_{\Omega} \nabla u \nabla v = \int_\Omega f v \hspace{2em} \forall v \in H_{0,D}^1.$$


In [1]:
import netgen.gui
%gui tk
from ngsolve import *
from mesh_util import uniform_1d_mesh

**Note (DD):** to generate a regular mesh with the boundary condition names set correctly
 just remove the maxh value from unit_square.GenerateMesh, then in Netgen refine it a couple times and save it.
 
 **Note (DD):** We can load geometry files but I'm not sure how to save them.

In [2]:

mesh = Mesh(uniform_1d_mesh())

In [3]:
mesh.nv, mesh.ne

(11, 10)

In [4]:

fes = H1(mesh, order=1, dirichlet="bottom|top|left|right")
fes.ndof  # number of unknowns in this space


11

In [5]:
u = fes.TrialFunction()  # symbolic object
v = fes.TestFunction()   # symbolic object
gfu = GridFunction(fes)  # solution 

In [6]:
a = BilinearForm(fes, symmetric=True)
a += SymbolicBFI(grad(u)*grad(v))
a.Assemble()

f = LinearForm(fes)
f += SymbolicLFI(x*v)
f.Assemble()


In [7]:
print(a.mat)

Row 0:   0: 10
Row 1:   0: -10   1: 20
Row 2:   1: -10   2: 20
Row 3:   2: -10   3: 20
Row 4:   3: -10   4: 20
Row 5:   4: -10   5: 20
Row 6:   5: -10   6: 20
Row 7:   6: -10   7: 20
Row 8:   7: -10   8: 20
Row 9:   8: -10   9: 20
Row 10:   9: -10   10: 10



I tried to get NGSolve to generate a matrix similar to the one in Claes Johnson 1.4, but it always puts in rows for the boundary elements and the diagonal elements have a different pattern.  

This is because the matrix in Claes Johnson doesn't include elements corresponding to Dirichlet boundary conditions, whereas NGSolve assumes Neumann boundary conditions when constructing the matrix.  The boundary constraints are taken into account by the Inverse (FreeDofs())

#### 6. Solve the system:

In [8]:
#gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs(), inverse='umfpack')*f.vec
#gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs(), inverse='sparsecholesky')*f.vec
gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs()) * f.vec
Draw(gfu)

To be able to visualize one-dimensional meshes and solution activate Show edges in the menu View > Viewing options > Mesh.

The Dirichlet boundary condition constrains some degrees of freedom. The argument ```fes.FreeDofs()``` indicates that only the remaining "free" degrees of freedom should participate in the linear solve.  We always need to specify it if we have Dirichlet boundary conditions.

You can examine the coefficient vector of solution if needed:


In [9]:
import matplotlib.pyplot as plt
plt.plot(gfu.vec)